In [38]:
#Get a list of PDB that contains histones from 'text.csv'
#cut -f1 text.tsv | uniq | awk '{print tolower($0)}' | sort

#text.tsv should be sorted by pdb and then uniprot name
#it MUST have 'NA' in uniprot and name blanks

In [39]:
#!/usr/bin/env python 3


PATH = "/net/pan1/interactomes/pipeline/Interactome/Workflow/Interfaces/"
CHAIN_FILE = "text.tsv"
PDB_LIST = "pdbList.txt"

In [40]:
def file_check(file):
    try:
        open(file, "r")
        return 1
    except IOError:
        print("Error: " + file + " does not appear to exist.")
        return 0

In [41]:
def get_files(pdbList, mFiles, iFiles):
    with open(pdbList, 'r') as pfh:
        pfh.readline()
        
        for line in pfh:
            line = line.strip()
            folder = line[1] + line[2] #folder name in ". . . Interfaces/"
            mFiles.append(PATH + folder + '/' + line + '_chain_protein_mapping.tab')
            iFiles.append(PATH + folder + '/' + line + '_atomic_contacts_5.0A.tab')

In [42]:
#The format of the dictionary is: AlexChain: myChain#UNIPROT#name
#Example: 'G': 'E#P02302#Histone H3.3C'

def get_chain_dictionaries(cFile, dictionary):  
    with open(cFile, 'r') as cfh:
        cfh.readline() #skips header
        
        for cLine in cfh:
            fields = cLine.strip().split('\t')
            pdb = fields[0]
            folder = pdb[1] + pdb[2] #folder name in ". . . Interfaces/"        
            
            tempDict = {}
            
            mappingFile = PATH + folder + '/' + pdb + '_chain_protein_mapping.tab'
            
            try:
                
                with open(mappingFile, 'r') as mfh:
                    mfh.readline()
                    
                    for mLine in mfh:
                        chainPair = mLine.split('\t', 2)
                        alexChain = chainPair[0]
                        myChain = chainPair[1]

                        tempDict[alexChain] = myChain
                
                dictionary[pdb] = {}
                pdbTemp = pdb

                while(pdbTemp == pdb):                    
                    chain = fields[1]
                    uniprot = fields[2]
                    name = fields[3]

                    if(not(uniprot == 'NA' and name == 'NA')): #prevents from wasting time on nucleotide chains
                        try:
                            alexChain = list(tempDict.keys())[list(tempDict.values()).index(chain)]
                            dictionary[pdb][alexChain] = str(tempDict[alexChain]) + '#' + uniprot + '#' + name
                        except ValueError:
                            pass
                            #print(ValueError)
                            
                    cLine = cfh.readline()
                    
                    fields = cLine.strip().split('\t')
                    pdbTemp = fields[0]
            except IOError:    
                #print("Error: " + mappingFile + " does not appear to exist.")
                pdbTemp = pdb
                
                while(pdbTemp == pdb):
                    cLine = cfh.readline()
                    fields = cLine.strip().split('\t', 1)
                    pdbTemp = fields[0]

In [43]:
def main():

    chainDictionary = {}
    chainDictionary['pdb'] = {}
    
    get_chain_dictionaries(CHAIN_FILE, chainDictionary)
    print(chainDictionary)

In [44]:
if __name__ == "__main__":
    main()

{'pdb': {}, '1a7w': {'A': 'A#P19267#DNA-binding protein HMf-2'}, '1aoi': {'I': 'G#P06897#Histone H2A type 1', 'F': 'D#P02281#Histone H2B 1.1', 'J': 'H#P02281#Histone H2B 1.1', 'C': 'A#P02302#Histone H3.3C', 'G': 'E#P02302#Histone H3.3C', 'D': 'B#P62799#Histone H4', 'H': 'F#P62799#Histone H4'}, '1b67': {'B': 'B#P48781#DNA-binding protein HMf-1'}, '1bh8': {'A': 'A#Q15543#Transcription initiation factor TFIID subunit 13'}, '1bh9': {'A': 'A#Q15543#Transcription initiation factor TFIID subunit 13'}, '1c3p': {'A': 'A#O67135#NA'}, '1c3r': {'B': 'B#O67135#NA'}, '1cm0': {'A': 'B#Q92831#Histone acetyltransferase KAT2B'}, '1dp7': {'B': 'P#P22670#MHC class II regulatory factor RFX1'}, '1e6i': {'B': 'P#P02309#Histone H4'}, '1eqz': {'C': 'A#P02263#Histone H2A-IV', 'G': 'E#P02263#Histone H2A-IV', 'D': 'B#P0C1H5#Histone H2B 7', 'H': 'F#P0C1H5#Histone H2B 7', 'E': 'C#P84229#Histone H3.2', 'I': 'G#P84229#Histone H3.2', 'F': 'D#P62801#Histone H4', 'J': 'H#P62801#Histone H4'}, '1f66': {'E': 'C#P0C0S5#Hist